In [195]:
import pandas as pd
import seaborn as sns
import random
import numpy as np
import math
wine = pd.read_csv('wine.csv')

In [196]:
wine.head(5)

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [197]:
class ProcessingData:
    
    @staticmethod
    def shuffle(base):
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalize(base):
        base.iloc[:,1:] = (base.iloc[:,1:]-base.iloc[:,1:].min())/(base.iloc[:,1:].max()-base.iloc[:,1:].min())

    @staticmethod
    def devide(base):
        n = int(len(base)*0.7)
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

In [198]:
wineEdit = wine.copy()

ProcessingData.shuffle(wineEdit)
ProcessingData.normalize(wineEdit)
trening_data,validate_data = ProcessingData.devide(wineEdit)

In [199]:
class Bayes:
    
    @staticmethod
    def recoverTreningData(data):
        sortedData = []
        sort = data.groupby(data.Wine)
        sortedData.append(sort.get_group(1))
        sortedData.append(sort.get_group(2))
        sortedData.append(sort.get_group(3))

        return sortedData

    @staticmethod
    def meanMaker(sortedData):
        meanTable = []
        for j in range(len(sortedData)):
            meanTable.append(sortedData[j].loc[:,sortedData[j].columns != "Wine"].mean(numeric_only=True).tolist())
        return meanTable

    @staticmethod
    def standardDeviation(sortedData):
        deviationTable = []
        for j in range(len(sortedData)):
            deviationTable.append(sortedData[j].loc[:,sortedData[j].columns != "Wine"].std(numeric_only=True).tolist())
        return deviationTable

    @staticmethod
    def gauss(validate_data,meanTable,deviationTable):
        correct = 0
        varieties = ["1","2","3"]
        for i in range(len(validate_data)):
            trueName = str(int(validate_data.iloc[i].Wine))
            points = []
            for j in range(len(sortedData)):
                g = 1
                for k in range(1,len(sortedData[j].columns)):
                    g *= (1/(((2*math.pi)**(1/2))*deviationTable[j][k-1])*math.exp(-((validate_data.iloc[i][k]-meanTable[j][k-1])**2)/(2*(deviationTable[j][k-1])**2)))
                points.append(g)
            guessedName = varieties[np.argmax(points)]

            if guessedName == trueName:
                correct+=1
        return "Accuracy {:.2f}% => {}/{}".format(correct*100/len(validate_data),correct,len(validate_data))

In [200]:
sortedData = Bayes.recoverTreningData(trening_data)
meanTable = Bayes.meanMaker(sortedData)
deviationTable = Bayes.standardDeviation(sortedData)
Bayes.gauss(validate_data,meanTable,deviationTable)

'Accuracy 96.30% => 52/54'